In [1]:
!wandb login

wandb: Currently logged in as: 202316187 (sergom2). Use `wandb login --relogin` to force relogin


In [2]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import wandb
import time

# Configuración inicial de wandb
wandb.init(project="ML2-img-class", name="Modelo1Grande", config={
    "learning_rate": 0.0001,
    "architecture": "resnet50",
    "dataset": "Tu Dataset Local",
    "unfreezed_layers": 10,  # Número de capas a descongelar
    "epochs": 25,
})
config = wandb.config

# Transformaciones
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=128, shuffle=False)

model = models.resnet50(pretrained=True)

# Descongelar las últimas 10 capas
children = list(model.children())
num_children = len(children)
for i in range(num_children - 10, num_children):
    for param in children[i].parameters():
        param.requires_grad = True

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct / total

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        
        start_time_epoch = time.time()  # Tiempo de inicio de la época
        
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)
            
            if i % 10 == 0:  # Imprime cada 10 lotes
                print(f'Época {epoch+1}, Lote {i+1}, Pérdida del Lote: {loss.item()}')
        
        end_time_epoch = time.time()  # Tiempo al final de la época
        epoch_duration = end_time_epoch - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f"Tiempo estimado restante: {estimated_time_left/60:.2f} minutos")
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = running_accuracy / len(train_loader.dataset)
        
        # Validación
        model.eval()
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * inputs.size(0)
                val_running_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)
                
        val_epoch_loss = val_running_loss / len(valid_loader.dataset)
        val_epoch_accuracy = val_running_accuracy / len(valid_loader.dataset)
        
        scheduler.step()
        
        # Log a W&B
        wandb.log({'train_loss': epoch_loss, 'train_accuracy': epoch_accuracy,
                   'val_loss': val_epoch_loss, 'val_accuracy': val_epoch_accuracy})
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_epoch_accuracy:.4f}')
               
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)
torch.save(model.state_dict(), 'model_1_grande.pth')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 202316187 (sergom2). Use `wandb login --relogin` to force relogin


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Época 1, Lote 1, Pérdida del Lote: 2.7169811725616455
Época 1, Lote 11, Pérdida del Lote: 1.1506268978118896
Época 1, Lote 21, Pérdida del Lote: 0.81302410364151
Tiempo estimado restante: 360.44 minutos
Epoch 1/25, Train Loss: 1.3586, Train Accuracy: 0.6238, Val Loss: 0.5721, Val Accuracy: 0.7807
Época 2, Lote 1, Pérdida del Lote: 0.5070626139640808
Época 2, Lote 11, Pérdida del Lote: 0.48501741886138916
Época 2, Lote 21, Pérdida del Lote: 0.4069483280181885
Tiempo estimado restante: 319.35 minutos
Epoch 2/25, Train Loss: 0.4782, Train Accuracy: 0.8533, Val Loss: 0.3470, Val Accuracy: 0.8853
Época 3, Lote 1, Pérdida del Lote: 0.48300760984420776
Época 3, Lote 11, Pérdida del Lote: 0.35647052526474
Época 3, Lote 21, Pérdida del Lote: 0.2899104654788971
Tiempo estimado restante: 338.78 minutos
Epoch 3/25, Train Loss: 0.3480, Train Accuracy: 0.8918, Val Loss: 0.2514, Val Accuracy: 0.9240
Época 4, Lote 1, Pérdida del Lote: 0.27938735485076904
Época 4, Lote 11, Pérdida del Lote: 0.217122495

In [2]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import wandb

# Configuración de Weights and Biases
wandb.init(project="ML2-img-class", name="densenet121_med", config={
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 64,
    "architecture": "densenet121"
})
config = wandb.config

# Transformaciones de datos
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Carga de datos
train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=config.batch_size, shuffle=False)

# Modelo
model = models.densenet121(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, len(train_data.classes))

# Configuración de entrenamiento
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

# Función de entrenamiento
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        total_loss, total_accuracy = 0, 0
        start_time_epoch = time.time()

        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()

            if i % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {i+1}, Batch Loss: {loss.item()}')

        scheduler.step()

        epoch_loss = total_loss / len(train_loader.dataset)
        epoch_accuracy = total_accuracy / len(train_loader.dataset)
        val_loss, val_accuracy = evaluate(model, valid_loader, criterion)
        wandb.log({'train_loss': epoch_loss, 'train_accuracy': epoch_accuracy, 'val_loss': val_loss, 'val_accuracy': val_accuracy})

        end_time_epoch = time.time()
        epoch_duration = end_time_epoch - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f'Estimated time remaining: {estimated_time_left/60:.2f} minutes')

    return model

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_accuracy = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()
    return total_loss / len(loader.dataset), total_accuracy / len(loader.dataset)

# Entrenamiento del modelo
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=config.epochs)
torch.save(trained_model.state_dict(), 'densenet121_model.pth')


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Batch 1, Batch Loss: 2.9709556102752686
Epoch 1, Batch 11, Batch Loss: 2.269395112991333
Epoch 1, Batch 21, Batch Loss: 2.030111789703369
Epoch 1, Batch 31, Batch Loss: 1.823324203491211
Epoch 1, Batch 41, Batch Loss: 1.543107271194458
Estimated time remaining: 47.68 minutes
Epoch 2, Batch 1, Batch Loss: 1.2467236518859863
Epoch 2, Batch 11, Batch Loss: 1.1343291997909546
Epoch 2, Batch 21, Batch Loss: 1.1605753898620605
Epoch 2, Batch 31, Batch Loss: 0.9497525691986084
Epoch 2, Batch 41, Batch Loss: 0.8718527555465698
Estimated time remaining: 44.69 minutes
Epoch 3, Batch 1, Batch Loss: 0.9105361104011536
Epoch 3, Batch 11, Batch Loss: 0.9864692091941833
Epoch 3, Batch 21, Batch Loss: 0.725793719291687
Epoch 3, Batch 31, Batch Loss: 0.829488217830658
Epoch 3, Batch 41, Batch Loss: 0.7449735999107361
Estimated time remaining: 39.54 minutes
Epoch 4, Batch 1, Batch Loss: 0.6305000185966492
Epoch 4, Batch 11, Batch Loss: 0.7752729654312134
Epoch 4, Batch 21, Batch Loss: 0.9353387

In [3]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import wandb

# Configuración de Weights and Biases
wandb.init(project="ML2-img-class", name="efficientnet_b0_med",config={
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 64,
    "architecture": "efficientnet_b0"
})
config = wandb.config

# Transformaciones de datos
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Carga de datos
train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=config.batch_size, shuffle=False)

# Modelo
model = models.efficientnet_b0(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(train_data.classes))

# Configuración de entrenamiento
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

# Función de entrenamiento
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        total_loss, total_accuracy = 0, 0
        start_time_epoch = time.time()

        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()

            if i % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {i+1}, Batch Loss: {loss.item()}')

        scheduler.step()

        epoch_loss = total_loss / len(train_loader.dataset)
        epoch_accuracy = total_accuracy / len(train_loader.dataset)
        val_loss, val_accuracy = evaluate(model, valid_loader, criterion)
        wandb.log({'train_loss': epoch_loss, 'train_accuracy': epoch_accuracy, 'val_loss': val_loss, 'val_accuracy': val_accuracy})

        end_time_epoch = time.time()
        epoch_duration = end_time_epoch - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f'Estimated time remaining: {estimated_time_left/60:.2f} minutes')

    return model

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_accuracy = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()
    return total_loss / len(loader.dataset), total_accuracy / len(loader.dataset)

# Entrenamiento del modelo
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=config.epochs)
torch.save(trained_model.state_dict(), 'efficientnet_b0_model.pth')


train_accuracy,▁▇▇███████
train_loss,█▃▂▁▁▁▁▁▁▁
val_accuracy,▁▆▇▇██████
val_loss,█▄▂▂▁▁▁▁▁▁
train_accuracy,0.82982
train_loss,0.63629
val_accuracy,0.872
val_loss,0.47158


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\sergi/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 22.4MB/s]


Epoch 1, Batch 1, Batch Loss: 2.7310023307800293
Epoch 1, Batch 11, Batch Loss: 2.4098026752471924
Epoch 1, Batch 21, Batch Loss: 2.118462324142456
Epoch 1, Batch 31, Batch Loss: 1.7233350276947021
Epoch 1, Batch 41, Batch Loss: 1.486593246459961
Estimated time remaining: 30.03 minutes
Epoch 2, Batch 1, Batch Loss: 1.3617490530014038
Epoch 2, Batch 11, Batch Loss: 1.1080161333084106
Epoch 2, Batch 21, Batch Loss: 1.1564067602157593
Epoch 2, Batch 31, Batch Loss: 1.1161137819290161
Epoch 2, Batch 41, Batch Loss: 1.0970815420150757
Estimated time remaining: 24.88 minutes
Epoch 3, Batch 1, Batch Loss: 1.1628406047821045
Epoch 3, Batch 11, Batch Loss: 1.0470322370529175
Epoch 3, Batch 21, Batch Loss: 0.9121251702308655
Epoch 3, Batch 31, Batch Loss: 0.9395942687988281
Epoch 3, Batch 41, Batch Loss: 0.9646850228309631
Estimated time remaining: 23.58 minutes
Epoch 4, Batch 1, Batch Loss: 0.7534129619598389
Epoch 4, Batch 11, Batch Loss: 0.9177544116973877
Epoch 4, Batch 21, Batch Loss: 0.791

In [4]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import wandb

# Configuración de Weights and Biases
wandb.init(project="ML2-img-class", name="mobilnetv2_med",config={
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 64,
    "architecture": "mobilenet_v2"
})
config = wandb.config

# Transformaciones de datos
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Carga de datos
train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=config.batch_size, shuffle=False)

# Modelo
model = models.mobilenet_v2(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier[1] = nn.Linear(model.last_channel, len(train_data.classes))

# Configuración de entrenamiento
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

# Función de entrenamiento
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        total_loss, total_accuracy = 0, 0
        start_time_epoch = time.time()

        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()

            if i % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {i+1}, Batch Loss: {loss.item()}')

        scheduler.step()

        epoch_loss = total_loss / len(train_loader.dataset)
        epoch_accuracy = total_accuracy / len(train_loader.dataset)
        val_loss, val_accuracy = evaluate(model, valid_loader, criterion)
        wandb.log({'train_loss': epoch_loss, 'train_accuracy': epoch_accuracy, 'val_loss': val_loss, 'val_accuracy': val_accuracy})

        end_time_epoch = time.time()
        epoch_duration = end_time_epoch - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f'Estimated time remaining: {estimated_time_left/60:.2f} minutes')

    return model

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_accuracy = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()
    return total_loss / len(loader.dataset), total_accuracy / len(loader.dataset)

# Entrenamiento del modelo
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=config.epochs)
torch.save(trained_model.state_dict(), 'mobilenet_v2_model.pth')


train_accuracy,▁▆▇███████
train_loss,█▄▂▁▁▁▁▁▁▁
val_accuracy,▁▄▆▆▇▇▇▇██
val_loss,█▄▂▂▁▁▁▁▁▁
train_accuracy,0.79464
train_loss,0.74009
val_accuracy,0.82933
val_loss,0.63816


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\sergi/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 42.4MB/s]


Epoch 1, Batch 1, Batch Loss: 2.821324110031128
Epoch 1, Batch 11, Batch Loss: 2.1060397624969482
Epoch 1, Batch 21, Batch Loss: 1.6546361446380615
Epoch 1, Batch 31, Batch Loss: 1.1582417488098145
Epoch 1, Batch 41, Batch Loss: 1.2890992164611816
Estimated time remaining: 17.38 minutes
Epoch 2, Batch 1, Batch Loss: 0.997840940952301
Epoch 2, Batch 11, Batch Loss: 0.887863278388977
Epoch 2, Batch 21, Batch Loss: 0.8057731986045837
Epoch 2, Batch 31, Batch Loss: 0.8361971974372864
Epoch 2, Batch 41, Batch Loss: 0.6521124839782715
Estimated time remaining: 14.98 minutes
Epoch 3, Batch 1, Batch Loss: 0.6037158966064453
Epoch 3, Batch 11, Batch Loss: 0.6404091119766235
Epoch 3, Batch 21, Batch Loss: 0.6929765343666077
Epoch 3, Batch 31, Batch Loss: 0.8236346244812012
Epoch 3, Batch 41, Batch Loss: 0.651573896408081
Estimated time remaining: 13.36 minutes
Epoch 4, Batch 1, Batch Loss: 0.5225253105163574
Epoch 4, Batch 11, Batch Loss: 0.6342203617095947
Epoch 4, Batch 21, Batch Loss: 0.69890

In [9]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import wandb

# Configuración de Weights and Biases
wandb.init(project="ML2-img-class", name="shufflenet_v2_med",config={
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 64,
    "architecture": "shufflenet_v2_x1_0"
})
config = wandb.config

# Transformaciones de datos
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Carga de datos
train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=config.batch_size, shuffle=False)

# Modelo
model = models.shufflenet_v2_x1_0(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))

# Configuración de entrenamiento
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

# Función de entrenamiento
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        total_loss, total_accuracy = 0, 0
        start_time_epoch = time.time()

        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()

            if i % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {i+1}, Batch Loss: {loss.item()}')

        scheduler.step()

        epoch_loss = total_loss / len(train_loader.dataset)
        epoch_accuracy = total_accuracy / len(train_loader.dataset)
        val_loss, val_accuracy = evaluate(model, valid_loader, criterion)
        wandb.log({'train_loss': epoch_loss, 'train_accuracy': epoch_accuracy, 'val_loss': val_loss, 'val_accuracy': val_accuracy})

        end_time_epoch = time.time()
        epoch_duration = end_time_epoch - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f'Estimated time remaining: {estimated_time_left/60:.2f} minutes')

    return model

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_accuracy = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()
    return total_loss / len(loader.dataset), total_accuracy / len(loader.dataset)

# Entrenamiento del modelo
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=config.epochs)
torch.save(trained_model.state_dict(), 'shufflenet_v2_x1_0_model.pth')


train_accuracy,▁▆▇▇▇█████
train_loss,█▃▂▁▁▁▁▁▁▁
val_accuracy,▁▆▇▆▇▇███▇
val_loss,█▃▂▂▁▁▁▁▁▁
train_accuracy,0.8392
train_loss,0.52859
val_accuracy,0.87733
val_loss,0.37016


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to C:\Users\sergi/.cache\torch\hub\checkpoints\shufflenetv2_x1-5666bf0f80.pth
100%|██████████| 8.79M/8.79M [00:00<00:00, 38.2MB/s]


Epoch 1, Batch 1, Batch Loss: 2.706932783126831
Epoch 1, Batch 11, Batch Loss: 2.6652212142944336
Epoch 1, Batch 21, Batch Loss: 2.629255533218384
Epoch 1, Batch 31, Batch Loss: 2.59684681892395
Epoch 1, Batch 41, Batch Loss: 2.562190532684326
Estimated time remaining: 9.54 minutes
Epoch 2, Batch 1, Batch Loss: 2.5428006649017334
Epoch 2, Batch 11, Batch Loss: 2.524078607559204
Epoch 2, Batch 21, Batch Loss: 2.4842708110809326
Epoch 2, Batch 31, Batch Loss: 2.475432872772217
Epoch 2, Batch 41, Batch Loss: 2.47778058052063
Estimated time remaining: 5.93 minutes
Epoch 3, Batch 1, Batch Loss: 2.4666359424591064
Epoch 3, Batch 11, Batch Loss: 2.4213998317718506
Epoch 3, Batch 21, Batch Loss: 2.386531114578247
Epoch 3, Batch 31, Batch Loss: 2.3473312854766846
Epoch 3, Batch 41, Batch Loss: 2.40496563911438
Estimated time remaining: 5.34 minutes
Epoch 4, Batch 1, Batch Loss: 2.310044050216675
Epoch 4, Batch 11, Batch Loss: 2.371171236038208
Epoch 4, Batch 21, Batch Loss: 2.2973647117614746
E